In [ ]:
import os

import torch

from CAM.cam_model import CNN, ResNetBackbone, fit_sgd, visualize_cam
from data import OxfordPetDataset, create_dataloaders

ImportError: cannot import name 'CAMWrapper' from 'CAM.cam_model' (/Users/dogukanturkoz/Documents/master/DL/cw2/ADL_CW2/CAM/cam_model.py)

In [2]:
## HYPERPARAMETERS
classification_mode = "breed"
batch_size = 32

model_type = "Res"  ## CNN, Res
model_dir = os.path.join("checkpoints", "CAM")
train_mode = True  # if False, will use trained local mode

num_epochs = 50

In [3]:
loss_function = torch.nn.CrossEntropyLoss()

In [ ]:
device = torch.device(
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
minibatch_size = 32

dataset = OxfordPetDataset().prepare_dataset()
train_loader, val_loader, test_loader = create_dataloaders(
    batch_size=batch_size, target_type=classification_mode
)

Images already downloaded: oxford_pet_data/images.tar.gz
Annotations already downloaded: oxford_pet_data/annotations.tar.gz
Dataset prepared with 35 classes.
Dataset split complete: training (70.0%), validation (15.0%), testing (15.0%)


In [5]:
### MODEL INIT
model_mode = classification_mode  ## species, breed
model_name = f"{model_type}_{model_mode}"
print(f"Using model {model_name}")

model_path = os.path.join(model_dir, model_name)

num_out = 37 if model_mode == "breed" else 2

### TRAIN MODEL
if train_mode:
    model_train = (
        ResNetBackbone(num_classes=num_out, pretrained=True)
        if "Res" in model_name
        else CNN(out_channels=256, num_classes=num_out)
    )
    fit_sgd(
        model_train,
        train_loader,
        classification_mode,
        num_epochs,
        3e-4,
        batch_size,
        loss_function,
        model_path,
        device=device,
    )

### TESTING MODEL
print(f"TESTING MODEL {model_path}.pt")
model_test = (
    ResNetBackbone(num_classes=num_out, pretrained=False)
    if "Res" in model_name
    else CNN(out_channels=256, num_classes=num_out)
)
model_test.load_state_dict(torch.load(f"{model_path}.pt"))

model_test.to(device)
model_test.eval()


visualize_cam(
    model_test, train_loader, model_mode, device
)  # Use loaded model on training dataset to show CAM

Using model Res_breed
<Training Start>
Epoch:1/50,Accuracy: 0.2872299382716049,1489/5184,Loss:2.9725952825428528


KeyboardInterrupt: 

In [ ]:
# from evaluation import evaluate_model


# _, val_loader, _ = custom_loader.create_dataloaders(dataset, batch_size, target_type="segmentation")

# cam_wrapper = CAMWrapper(model_test)
# cam_wrapper.to("cpu")
# evaluate_model(cam_wrapper, val_loader, 8, model_name, device = "cpu")

Dataset split complete: training (70.0%), validation (15.0%), testing (15.0%)
Evaluating model...
Mean IoU: 1.8222098214285714
Mean F1 Score: 1.8252898659024919
